# TensorFlow: Variable

TensorFlow 모델 생성 시 가장 많이 신경써야 하는 것이 Variable입니다.
자세히 Variable에 대해 알아보도록 합시다.

In [1]:
import tensorflow as tf

## 1. tf.Variable vs. tf.get_variable

[StackOverFlow: Difference between Variable and get_variable in TensorFlow](http://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow)
> tf.Variable is a class.

> There are several ways to create tf.Variable including 
> - tf.Variable.__init__
> - tf.get_variable.

[TensorFlow API_guide: Variable](https://www.tensorflow.org/api_guides/python/state_ops)
- Class definitions and related functions for **tf.Variable** are introduced.

### 1.1. tf.Variable
> - tf.Variable.__init__: Creates a new variable with *initial_value*.
> - [TensorFlow API document: tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)

In [2]:
w = tf.Variable(initial_value=tf.truncated_normal(shape=[4, 4], mean=0.0, stddev=0.1))

In [3]:
print(w)

Tensor("Variable/read:0", shape=(4, 4), dtype=float32)


In [4]:
w = tf.Variable(initial_value=tf.truncated_normal(shape=[4, 4], mean=0.0, stddev=0.1))

In [5]:
print(w)

Tensor("Variable_1/read:0", shape=(4, 4), dtype=float32)


w를 선언할 때마다, Variable의 이름이 변화하는 것을 확인할 수 있다.
- Variable/read:0
- Variable_1/read:0

In [6]:
# Variable에 이름 추가 (name='testVariable')
w_1 = tf.Variable(initial_value=tf.truncated_normal(shape=[4, 4,], mean=0.0, stddev=0.1), name='testVariable')

In [7]:
print(w_1)

Tensor("testVariable/read:0", shape=(4, 4), dtype=float32)


In [8]:
w_1 = tf.Variable(initial_value=tf.truncated_normal(shape=[4, 4,], mean=0.0, stddev=0.1), name='testVariable')

In [9]:
print(w_1)

Tensor("testVariable_1/read:0", shape=(4, 4), dtype=float32)


Every time you declare w, you can see that the name of the *tf.Variable* changes.
- testVariable/read:0
- testVariable_1/read:0

### 1.2. tf.get_variable
> - tf.get_variable: Gets an existing variable with these parameters or create a new one.
> - [TensorFlow API document: tf.get_variable](https://www.tensorflow.org/api_docs/python/tf/get_variable)

In [10]:
w_2 = tf.get_variable(name='testGetVar', shape=[4, 5], 
                      initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))

In [11]:
print(w_2)

Tensor("testGetVar/read:0", shape=(4, 5), dtype=float32)


In [12]:
# ## You can see ValueError...
# # "ValueError: Variable testGetVar already exists, disallowed."
# w_2 = tf.get_variable(name='testGetVar', shape=[4, 4], 
#                       initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))

Why does that error appear?
- Because tf.Variable named "testGetVar" already exists.
- You should define variable scope and do setting reuse_variables(). (See the below!)
- See [TensorFlow Programming Guide: Sharing Variables](https://www.tensorflow.org/programmers_guide/variable_scope)

In [13]:
with tf.variable_scope("foo") as scope:
    w_3 = tf.get_variable("testGetVar2", shape=[4, 4], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    scope.reuse_variables()
    w_4 = tf.get_variable("testGetVar2", shape=[4, 4], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
assert w_4 is w_3

In [14]:
print(w_3)
print(w_4)

Tensor("foo/testGetVar2/read:0", shape=(4, 4), dtype=float32)
Tensor("foo/testGetVar2/read:0", shape=(4, 4), dtype=float32)


Anyway,
Many researchers (and I) **STRONGLY RECOMMEND** to use *tf.get_variable* instead of *tf.Variable*!

## 2. Variable scope

In [15]:
with tf.variable_scope("Global"):
    t_1 = tf.get_variable("T1", [2, 2])
    with tf.variable_scope("Asia"):
        t_2 = tf.get_variable("T2", [1])
        t_3 = tf.get_variable("T3", [1, 2])
        with tf.variable_scope("Korea"):
            t_4 = tf.get_variable("T4", [1, 2, 2])
            t_5 = tf.get_variable("T5", [2, 2, 2, 2])

In [16]:
print(t_1)

Tensor("Global/T1/read:0", shape=(2, 2), dtype=float32)


In [17]:
print(t_1.name)
print(t_2.name)
print(t_3.name)
print(t_4.name)
print(t_5.name)

Global/T1:0
Global/Asia/T2:0
Global/Asia/T3:0
Global/Asia/Korea/T4:0
Global/Asia/Korea/T5:0


- Note: [StackOverFlow: difference-of-name-scope-and-a-variable-scope-in-tensorflow](http://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow)

In [18]:
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

var1:0
my_scope/var2:0
my_scope/Add:0


In [19]:
with tf.variable_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # my_scope/var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

my_scope/var1:0
my_scope_1/var2:0
my_scope_1/Add:0
